In [ ]:
from utils.training_loop import set_seed, seed_worker, get_optimizer, train_one_epoch, evaluate_one_epoch
from utils.helper import save_imgs_df, get_std
from utils.transforms import get_transform
from utils.data import data_split, create_loader
from utils.model import get_model
from utils.training_loop import set_seed, seed_worker, get_optimizer, train_one_epoch, evaluate_one_epoch
from utils.checkpoint import save_best_model_weight, save_hparam, save_loss_log
from omegaconf import OmegaConf

import pandas as pd
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder

In [ ]:
def main(cfg):
    baseline_cfg = OmegaConf.load(cfg.baseline_cfg)
    transform = get_transform(cfg.transform.resize, cfg.transform.normalize.mean, cfg.transform.normalize.std)
    
    dataset = ImageFolder(cfg.data.test_dir, transform=transform)
    loader = create_loader(dataset, cfg.eval.batch_size, is_train=False, num_workers=4)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_model(cfg.models.resnet50, device)
    state_dict = torch.load(cfg.models.checkpoint.best_model_weight)
    model.load_state_dict(state_dict)
    model.eval()

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)
            preds = torch.argmax(logits, dim=1)
            print('--'*20)
            print(f"Batch {i+1}/{len(loader)}, Predicted: {preds}, Actual: {labels}")
            print(f'Accuracy: {torch.sum(preds == labels).item() / len(labels) * 100:.2f}%')

            






    



